# `TEST:` Decide whether a collection of rational numbers `u_i,v_i` is realizable by a quadratic map

### Introduction

In this script we test if there is any obstuction coming from our formulas for a collection of 14 rational numbers `u_i, v_i` to come from a quadratic self-map of the complex projective plane having a (forward) invariant line.

More precisely, let `[u_0,v_0,...,u_6,v_6]` be a collection of rational numbers and suppose there exists a quadratic map $f$ with non-degenerate fixed points $p_0,\ldots, p_6$, satisfying the following:

* `u_i, v_i` are the eigenvalues of $\mathrm{I}-Df(p_i)$,
* The points $p_0,\ldots,p_3$ lie away from the invariant line, and the points $p_4,p_5,p_6$ are on it,
* For i=4,5,6, the eigenvalue `u_i` corresponds to the direction tangent to the invariant line.

Then the coefficients `a,b,c_0,...,c_5` defining the map $f$ satisfy the equations given by `Eq, Rel, Form, gc` given below. Therefore, the ideals `I` and `Ielim` defined below define a non-empty affine variety. In particular we must have `J.dimension()` $\geq$ 0 (by convention the dimension of the empty variety is -1).

### Usage

**Input:**  
Provide a list of 14 rational numbers as follows:
```python
Input = [u_0,v_0,u_1,v_1,u_2,v_2,u_3,v_3,u_4,v_4,u_5,v_5,u_6,v_6],
```
or, if the variable `'trace_determinant_format'` is set to `'True'`, provide
```python
Input = [t_0,d_0,t_1,d_1,t_2,d_2,t_3,d_3,u_4,v_4,u_5,v_5,u_6,v_6],
```
where `t_i = u_i + v_i`, `d_i = u_i*v_i`.

**Output:**  
The program will return either
```python
Test passed = True
```
if `J.dimension() != -1` (namely, if J defines a non-empty affine variety), or
```python
Test passed = False
```
otherwise.

### Scope

The test is not passed if and only if the constant 1 belongs to the ideal J. In this case we can be certain that it is impossible for the values provided in `'Input'` to come from a quadratic map.

On the other hand, if the test is passed we only know that our equations do not provide an obstruction to realizability. Further analysis is needed to decide if the collection `'Input'` is truly realizable. However, a close look to the generators of the ideal `J` computed during the elimination process `J <- I.elimination_ideal(w_i,x,y)` provides a system of equations for the variables `a,b,c_i` which can usually be easily solved. Finding a solution to such system would confirm the realizability of the proposed spectra and would construct an explicit map that realizes them.

### The TEST

In [1]:
# import test_aux

Choose the format of the input:

In [2]:
# If True the format of the first 8 entries must be t_i,d_i instead of u_i,v_i
trace_determinant_format = True

Provide a list of 14 rational numbers according to the format chosen above, or try one of the following examples:
```python
Input = case_study
Input = degenerate_case
```

In [3]:
Input = [-4, 3, -3/5, -4/25, 4/3, 1/3, 9, -60, -2/3, -1/3, 2/3, -5/3, 1, 3]

In [4]:
def test_input(Input):
    message = 'Input must be a list of 14 rational numbers'
    assert isinstance(Input, list), message
    assert len(Input) == 14, message
    for i in range(14):
        assert isinstance(Input[i], sage.rings.integer.Integer) or isinstance(
            Input[i], sage.rings.rational.Rational), message
    message = 'Eigenvalues cannot be zero'
    if trace_determinant_format is False:
        for i in range(14):
            assert Input[i] != 0, message
    else:
        for i in range(4):
            assert Input[2 * i + 1] != 0, message
        for i in range(8, 14):
            assert Input[i] != 0, message
    return 'Input OK'

In [5]:
test_input(Input)

'Input OK'

In [6]:
def build_uv(Input):
    u = [0 for _ in range(7)]
    v = [0 for _ in range(7)]
    for i in range(7):
        u[i] = Input[2 * i]
        v[i] = Input[2 * i + 1]
    if trace_determinant_format is True:
        for i in range(4):
            u[i] = None
            v[i] = None
    return (u, v)

In [7]:
u, v = build_uv(Input)
u_4, u_5, u_6 = u[4:]
v_4, v_5, v_6 = v[4:]

In [8]:
def build_td(Input):
    t = [0 for _ in range(4)]
    d = [0 for _ in range(4)]
    if trace_determinant_format is False:
        for i in range(4):
            t[i] = Input[2*i] + Input[2*i + 1]
            d[i] = Input[2*i] * Input[2*i + 1]
    else:
        for i in range(4):
            t[i] = Input[2*i]
            d[i] = Input[2*i + 1]
    return (t, d)

In [9]:
t, d = build_td(Input)
t_0, t_1, t_2, t_3 = tuple(t)
d_0, d_1, d_2, d_3 = tuple(d)

In [10]:
# print variables function

In [11]:
print 'Input:'
for i in range(4):
    print 't_{} ='.format(i), eval('t_{}'.format(i))
    print 'd_{} ='.format(i), eval('d_{}'.format(i))
for i in range(4, 7):
    print 'u_{} ='.format(i), eval('u_{}'.format(i))
    print 'v_{} ='.format(i), eval('v_{}'.format(i))

Input:
t_0 = -4
d_0 = 3
t_1 = -3/5
d_1 = -4/25
t_2 = 4/3
d_2 = 1/3
t_3 = 9
d_3 = -60
u_4 = -2/3
v_4 = -1/3
u_5 = 2/3
v_5 = -5/3
u_6 = 1
v_6 = 3


In [12]:
R = QQ['w_0,w_1,w_2,w_3,x,y,c_0,c_1,c_2,c_3,c_4,c_5,a,b']
(w_0, w_1, w_2, w_3, x, y, c_0, c_1, c_2, c_3, c_4, c_5, a, b) = R.gens()

# Relations coming from Jacobi formula:
Rel = [0 for i in range(5)]
Rel[0] = a^2*d_3*x^2 + 2*a*b*d_3*x*y + b^2*d_3*y^2 + a^2*d_1*x + 2*a*d_1*x + 2*a*d_3*x + 2*b*d_3*y + d_1*x + d_3
Rel[1] = a^2*d_3*x^2 + 2*a*b*d_3*x*y + b^2*d_3*y^2 + b^2*d_2*y + 2*a*d_3*x + 2*b*d_2*y + 2*b*d_3*y + d_2*y + d_3
Rel[2] = a^2*b*d_1*d_2*d_3*x + a*b^2*d_1*d_2*d_3*y + a^2*d_0*d_1*d_3*x + a*b*d_0*d_2*d_3*x + a^2*d_1*d_2*d_3*x + a*b*d_1*d_2*d_3*x + a*b*d_0*d_1*d_3*y + b^2*d_0*d_2*d_3*y + a*b*d_1*d_2*d_3*y + b^2*d_1*d_2*d_3*y + a*b*d_0*d_1*d_2 + a*b*d_1*d_2*d_3 + a*d_0*d_1*d_3*x + a*d_0*d_2*d_3*x + a*d_1*d_2*d_3*x + b*d_0*d_1*d_3*y + b*d_0*d_2*d_3*y + b*d_1*d_2*d_3*y + a*d_0*d_1*d_2 + b*d_0*d_1*d_2 + a*d_0*d_1*d_3 + b*d_0*d_2*d_3 + a*d_1*d_2*d_3 + b*d_1*d_2*d_3 + d_0*d_1*d_2 + d_0*d_1*d_3 + d_0*d_2*d_3 + d_1*d_2*d_3
Rel[3] = a^2*b*d_1*d_2*d_3*t_0*x + a*b^2*d_1*d_2*d_3*t_0*y + a^2*d_1*d_2*d_3*t_0*x + a*b*d_1*d_2*d_3*t_0*x + a*b*d_0*d_2*d_3*t_1*x + a^2*d_0*d_1*d_3*t_2*x + a*b*d_1*d_2*d_3*t_0*y + b^2*d_1*d_2*d_3*t_0*y + b^2*d_0*d_2*d_3*t_1*y + a*b*d_0*d_1*d_3*t_2*y + a*b*d_1*d_2*d_3*t_0 + a*b*d_0*d_1*d_2*t_3 + a*d_1*d_2*d_3*t_0*x + a*d_0*d_2*d_3*t_1*x + a*d_0*d_1*d_3*t_2*x + b*d_1*d_2*d_3*t_0*y + b*d_0*d_2*d_3*t_1*y + b*d_0*d_1*d_3*t_2*y + a*d_1*d_2*d_3*t_0 + b*d_1*d_2*d_3*t_0 + b*d_0*d_2*d_3*t_1 + a*d_0*d_1*d_3*t_2 + a*d_0*d_1*d_2*t_3 + b*d_0*d_1*d_2*t_3 + d_1*d_2*d_3*t_0 + d_0*d_2*d_3*t_1 + d_0*d_1*d_3*t_2 + d_0*d_1*d_2*t_3
Rel[4] = a^4*b^2*d_1*d_2*d_3*x^2 + 2*a^3*b^3*d_1*d_2*d_3*x*y + a^2*b^4*d_1*d_2*d_3*y^2 + 2*a^4*b*d_1*d_2*d_3*x^2 + 2*a^3*b^2*d_1*d_2*d_3*x^2 + 4*a^3*b^2*d_1*d_2*d_3*x*y + 4*a^2*b^3*d_1*d_2*d_3*x*y + 2*a^2*b^3*d_1*d_2*d_3*y^2 + 2*a*b^4*d_1*d_2*d_3*y^2 + 2*a^3*b^2*d_1*d_2*d_3*x + a^4*d_0*d_1*d_3*x^2 + a^2*b^2*d_0*d_2*d_3*x^2 + a^4*d_1*d_2*d_3*x^2 + 4*a^3*b*d_1*d_2*d_3*x^2 + a^2*b^2*d_1*d_2*d_3*x^2 + 2*a^2*b^3*d_1*d_2*d_3*y + 2*a^3*b*d_0*d_1*d_3*x*y + 2*a*b^3*d_0*d_2*d_3*x*y + 2*a^3*b*d_1*d_2*d_3*x*y + 8*a^2*b^2*d_1*d_2*d_3*x*y + 2*a*b^3*d_1*d_2*d_3*x*y + a^2*b^2*d_0*d_1*d_3*y^2 + b^4*d_0*d_2*d_3*y^2 + a^2*b^2*d_1*d_2*d_3*y^2 + 4*a*b^3*d_1*d_2*d_3*y^2 + b^4*d_1*d_2*d_3*y^2 + 4*a^3*b*d_1*d_2*d_3*x + 4*a^2*b^2*d_1*d_2*d_3*x + 2*a^3*d_0*d_1*d_3*x^2 + 2*a^2*b*d_0*d_2*d_3*x^2 + 2*a^3*d_1*d_2*d_3*x^2 + 2*a^2*b*d_1*d_2*d_3*x^2 + 4*a^2*b^2*d_1*d_2*d_3*y + 4*a*b^3*d_1*d_2*d_3*y + 4*a^2*b*d_0*d_1*d_3*x*y + 4*a*b^2*d_0*d_2*d_3*x*y + 4*a^2*b*d_1*d_2*d_3*x*y + 4*a*b^2*d_1*d_2*d_3*x*y + 2*a*b^2*d_0*d_1*d_3*y^2 + 2*b^3*d_0*d_2*d_3*y^2 + 2*a*b^2*d_1*d_2*d_3*y^2 + 2*b^3*d_1*d_2*d_3*y^2 + a^2*b^2*d_0*d_1*d_2 + a^2*b^2*d_1*d_2*d_3 + 2*a^3*d_0*d_1*d_3*x + 2*a*b^2*d_0*d_2*d_3*x + 2*a^3*d_1*d_2*d_3*x + 8*a^2*b*d_1*d_2*d_3*x + 2*a*b^2*d_1*d_2*d_3*x + a^2*d_0*d_1*d_3*x^2 + a^2*d_0*d_2*d_3*x^2 + a^2*d_1*d_2*d_3*x^2 + 2*a^2*b*d_0*d_1*d_3*y + 2*b^3*d_0*d_2*d_3*y + 2*a^2*b*d_1*d_2*d_3*y + 8*a*b^2*d_1*d_2*d_3*y + 2*b^3*d_1*d_2*d_3*y + 2*a*b*d_0*d_1*d_3*x*y + 2*a*b*d_0*d_2*d_3*x*y + 2*a*b*d_1*d_2*d_3*x*y + b^2*d_0*d_1*d_3*y^2 + b^2*d_0*d_2*d_3*y^2 + b^2*d_1*d_2*d_3*y^2 + 2*a^2*b*d_0*d_1*d_2 + 2*a*b^2*d_0*d_1*d_2 + 2*a^2*b*d_1*d_2*d_3 + 2*a*b^2*d_1*d_2*d_3 + 4*a^2*d_0*d_1*d_3*x + 4*a*b*d_0*d_2*d_3*x + 4*a^2*d_1*d_2*d_3*x + 4*a*b*d_1*d_2*d_3*x + 4*a*b*d_0*d_1*d_3*y + 4*b^2*d_0*d_2*d_3*y + 4*a*b*d_1*d_2*d_3*y + 4*b^2*d_1*d_2*d_3*y + a^2*d_0*d_1*d_2 + 4*a*b*d_0*d_1*d_2 + b^2*d_0*d_1*d_2 + a^2*d_0*d_1*d_3 + b^2*d_0*d_2*d_3 + a^2*d_1*d_2*d_3 + 4*a*b*d_1*d_2*d_3 + b^2*d_1*d_2*d_3 + 2*a*d_0*d_1*d_3*x + 2*a*d_0*d_2*d_3*x + 2*a*d_1*d_2*d_3*x + 2*b*d_0*d_1*d_3*y + 2*b*d_0*d_2*d_3*y + 2*b*d_1*d_2*d_3*y + 2*a*d_0*d_1*d_2 + 2*b*d_0*d_1*d_2 + 2*a*d_0*d_1*d_3 + 2*b*d_0*d_2*d_3 + 2*a*d_1*d_2*d_3 + 2*b*d_1*d_2*d_3 + d_0*d_1*d_2 + d_0*d_1*d_3 + d_0*d_2*d_3 + d_1*d_2*d_3

# Traces and determinants of auxiliary vector field H
T_0 = t_0
T_1 = t_1*(a+1)
T_2 = t_2*(b+1)
D_0 = d_0
D_1 = d_1*(a+1)^2
D_2 = d_2*(b+1)^2

# Comparing with the known formulas for T_i, D_i we get these equations:
Eq = [0 for i in range(6)]
Eq[0] = T_0 - (-c_0-c_5)
Eq[1] = T_1 - (c_0+c_4-c_5)
Eq[2] = T_2 - (-c_0+c_1+c_5)
Eq[3] = D_0 - (-c_2*c_3+c_0*c_5)
Eq[4] = D_1 - (-c_1*c_3+c_2*c_3+c_0*c_4-c_0*c_5)
Eq[5] = D_2 - (c_2*c_3-c_2*c_4-c_0*c_5+c_1*c_5)

# Formulas for the invariant functions p,q,r,s (after lifting denominators)
p = u_4 + u_5 + u_6
q = u_4*u_5*u_6
r = v_4 + v_5 + v_6
s = u_4*(v_5 + v_6) + u_5*(v_4 + v_6) + u_6*(v_4 + v_5)

Form = [0 for i in range(4)]
Form[0] = p*(-c_3^2*c_2^2 - 3*a*c_0*b*c_5 + a*c_4*b*c_5 + b^3*c_3 - c_0^2*b^2 - c_0^2*c_5^2 - b^2*c_3*c_5 + 2*c_0^2*b*c_5 + a*c_0*b^2 + 2*a*c_0*c_5^2 - a*c_4*b^2 + c_0*c_4*b^2 - c_0*c_4*b*c_5 - c_1^2*c_3*c_5 + c_5*c_4*c_0*c_1 + 2*c_2*c_0*c_5*c_3 + c_2*c_4*c_1*c_3 - c_2*c_0*c_4^2 + c_1^2*c_3*b - 2*c_1*b^2*c_3 - 2*c_2*c_0*c_3*b - b*c_4*c_0*c_1 - c_2*c_4*c_3*b + 2*c_1*c_3*b*c_5 - c_1*c_5*c_0*a - c_1*c_5*c_4*a - c_2*c_3*c_1*a - 2*c_2*c_3*c_5*a + 2*c_2*c_0*c_4*a + c_1*c_4*b*a + 3*c_2*c_3*b*a + c_1*c_0*b*a + c_2*a^3 - c_5^2*a^2 - c_0*c_2*a^2 - 2*c_4*c_2*a^2 + b*c_5*a^2 + c_2*c_4^2*a + c_1*c_5*a^2 - c_1*b*a^2) - (-9*c_3^2*c_2^2 - a*c_0*b*c_5 + a*c_4*b*c_5 - c_0^2*b^2 - c_0^2*c_5^2 - c_4^2*b^2 + 3*b^2*c_3*c_5 + 2*c_0^2*b*c_5 - 4*c_5^2*c_3*b + 2*a*c_0*c_5^2 + 2*c_0*c_4*b^2 + c_4^2*b*c_5 - 3*c_0*c_4*b*c_5 - c_1^2*c_4*c_0 - c_5*c_4^2*c_1 - 5*c_1^2*c_3*c_5 + 3*c_5*c_4*c_0*c_1 - 4*c_2*c_0*c_1*c_3 + 10*c_2*c_0*c_5*c_3 + 7*c_2*c_4*c_1*c_3 - 4*c_2*c_4*c_5*c_3 + 4*c_1*c_5^2*c_3 + 4*c_2*c_0^2*c_4 - 5*c_2*c_0*c_4^2 + 2*c_1^2*c_3*b + c_4^2*c_1*b - 3*c_1*b^2*c_3 + c_1^3*c_3 + c_2*c_4^3 - 6*c_2*c_0*c_3*b - b*c_4*c_0*c_1 - 3*c_2*c_4*c_3*b + 2*c_1*c_3*b*c_5 - 3*c_1*c_5*c_0*a - c_1*c_5*c_4*a - 3*c_2*c_3*c_1*a - 6*c_2*c_3*c_5*a + 2*c_2*c_0*c_4*a - c_1*c_4*b*a + 9*c_2*c_3*b*a + c_1*c_0*b*a - c_5^2*a^2 + 3*c_0*c_2*a^2 - 3*c_4*c_2*a^2 + 2*c_2*c_4^2*a + 2*c_1*c_5*a^2 - a^2*c_1^2 - 4*a*c_2*c_0^2 + a*c_1^2*c_4 + a*c_1^2*c_0)
Form[1] = q*-(-a*b^2*c_0 + b^2*c_0^2 + a^2*b*c_1 - a*b*c_0*c_1 - a^3*c_2 + a^2*c_0*c_2 - b^3*c_3 + 2*b^2*c_1*c_3 - b*c_1^2*c_3 - 3*a*b*c_2*c_3 + 2*b*c_0*c_2*c_3 + a*c_1*c_2*c_3 + c_2^2*c_3^2 + a*b^2*c_4 - b^2*c_0*c_4 - a*b*c_1*c_4 + b*c_0*c_1*c_4 + 2*a^2*c_2*c_4 - 2*a*c_0*c_2*c_4 + b*c_2*c_3*c_4 - c_1*c_2*c_3*c_4 - a*c_2*c_4^2 + c_0*c_2*c_4^2 - a^2*b*c_5 + 3*a*b*c_0*c_5 - 2*b*c_0^2*c_5 - a^2*c_1*c_5 + a*c_0*c_1*c_5 + b^2*c_3*c_5 - 2*b*c_1*c_3*c_5 + c_1^2*c_3*c_5 + 2*a*c_2*c_3*c_5 - 2*c_0*c_2*c_3*c_5 - a*b*c_4*c_5 + b*c_0*c_4*c_5 + a*c_1*c_4*c_5 - c_0*c_1*c_4*c_5 + a^2*c_5^2 - 2*a*c_0*c_5^2 + c_0^2*c_5^2) - (c_0^2*c_1^2 - 4*c_0^3*c_2 + 4*c_1^3*c_3 - 18*c_0*c_1*c_2*c_3 - 27*c_2^2*c_3^2 - 2*c_0*c_1^2*c_4 + 12*c_0^2*c_2*c_4 + 18*c_1*c_2*c_3*c_4 + c_1^2*c_4^2 - 12*c_0*c_2*c_4^2 + 4*c_2*c_4^3 - 2*c_0^2*c_1*c_5 - 12*c_1^2*c_3*c_5 + 18*c_0*c_2*c_3*c_5 + 4*c_0*c_1*c_4*c_5 - 18*c_2*c_3*c_4*c_5 - 2*c_1*c_4^2*c_5 + c_0^2*c_5^2 + 12*c_1*c_3*c_5^2 - 2*c_0*c_4*c_5^2 + c_4^2*c_5^2 - 4*c_3*c_5^3)
Form[2] = r*-(-c_3^2*c_2^2 - 3*a*c_0*b*c_5 + a*c_4*b*c_5 + b^3*c_3 - c_0^2*b^2 - c_0^2*c_5^2 - b^2*c_3*c_5 + 2*c_0^2*b*c_5 + a*c_0*b^2 + 2*a*c_0*c_5^2 - a*c_4*b^2 + c_0*c_4*b^2 - c_0*c_4*b*c_5 - c_1^2*c_3*c_5 + c_5*c_4*c_0*c_1 + 2*c_2*c_0*c_5*c_3 + c_2*c_4*c_1*c_3 - c_2*c_0*c_4^2 + c_1^2*c_3*b - 2*c_1*b^2*c_3 - 2*c_2*c_0*c_3*b - b*c_4*c_0*c_1 - c_2*c_4*c_3*b + 2*c_1*c_3*b*c_5 - c_1*c_5*c_0*a - c_1*c_5*c_4*a - c_2*c_3*c_1*a - 2*c_2*c_3*c_5*a + 2*c_2*c_0*c_4*a + c_1*c_4*b*a + 3*c_2*c_3*b*a + c_1*c_0*b*a + c_2*a^3 - c_5^2*a^2 - c_0*c_2*a^2 - 2*c_4*c_2*a^2 + b*c_5*a^2 + c_2*c_4^2*a + c_1*c_5*a^2 - c_1*b*a^2) - (3*c_3^2*c_2^2 + 3*a*c_0*b*c_5 - a*c_4*b*c_5 + c_0^2*b^2 + 3*c_0^2*c_5^2 + b^2*c_3*c_5 - 4*c_0^2*b*c_5 - 4*a*c_0*c_5^2 - c_0*c_4*b^2 + 2*c_0*c_4*b*c_5 + 3*c_1^2*c_3*c_5 - 3*c_5*c_4*c_0*c_1 - 6*c_2*c_0*c_5*c_3 - 3*c_2*c_4*c_1*c_3 + 3*c_2*c_0*c_4^2 - 2*c_1^2*c_3*b + 2*c_1*b^2*c_3 + 4*c_2*c_0*c_3*b + 2*b*c_4*c_0*c_1 + 2*c_2*c_4*c_3*b - 4*c_1*c_3*b*c_5 + 2*c_1*c_5*c_0*a + 2*c_1*c_5*c_4*a + 2*c_2*c_3*c_1*a + 4*c_2*c_3*c_5*a - 4*c_2*c_0*c_4*a - c_1*c_4*b*a - 3*c_2*c_3*b*a - c_1*c_0*b*a + c_5^2*a^2 + c_0*c_2*a^2 + 2*c_4*c_2*a^2 - 2*c_2*c_4^2*a - c_1*c_5*a^2)
Form[3] = s*-(-c_3^2*c_2^2 - 3*a*c_0*b*c_5 + a*c_4*b*c_5 + b^3*c_3 - c_0^2*b^2 - c_0^2*c_5^2 - b^2*c_3*c_5 + 2*c_0^2*b*c_5 + a*c_0*b^2 + 2*a*c_0*c_5^2 - a*c_4*b^2 + c_0*c_4*b^2 - c_0*c_4*b*c_5 - c_1^2*c_3*c_5 + c_5*c_4*c_0*c_1 + 2*c_2*c_0*c_5*c_3 + c_2*c_4*c_1*c_3 - c_2*c_0*c_4^2 + c_1^2*c_3*b - 2*c_1*b^2*c_3 - 2*c_2*c_0*c_3*b - b*c_4*c_0*c_1 - c_2*c_4*c_3*b + 2*c_1*c_3*b*c_5 - c_1*c_5*c_0*a - c_1*c_5*c_4*a - c_2*c_3*c_1*a - 2*c_2*c_3*c_5*a + 2*c_2*c_0*c_4*a + c_1*c_4*b*a + 3*c_2*c_3*b*a + c_1*c_0*b*a + c_2*a^3 - c_5^2*a^2 - c_0*c_2*a^2 - 2*c_4*c_2*a^2 + b*c_5*a^2 + c_2*c_4^2*a + c_1*c_5*a^2 - c_1*b*a^2) - (18*c_3^2*c_2^2 + 2*c_0^2*c_5^2 - 2*c_0^2*b*c_5 + 4*c_5^2*c_3*b - 2*a*c_0*c_5^2 - c_4^2*b*c_5 + 3*c_0*c_4*b*c_5 + 2*c_1^2*c_4*c_0 + 2*c_5*c_4^2*c_1 + 10*c_1^2*c_3*c_5 - 6*c_5*c_4*c_0*c_1 + 8*c_2*c_0*c_1*c_3 - 20*c_2*c_0*c_5*c_3 - 14*c_2*c_4*c_1*c_3 + 8*c_2*c_4*c_5*c_3 - 8*c_1*c_5^2*c_3 - 8*c_2*c_0^2*c_4 + 10*c_2*c_0*c_4^2 - 2*c_1^2*c_3*b - c_4^2*c_1*b - 2*c_1^3*c_3 - 2*c_2*c_4^3 + 6*c_2*c_0*c_3*b + b*c_4*c_0*c_1 + 3*c_2*c_4*c_3*b - 2*c_1*c_3*b*c_5 + 3*c_1*c_5*c_0*a + c_1*c_5*c_4*a + 3*c_2*c_3*c_1*a + 6*c_2*c_3*c_5*a - 2*c_2*c_0*c_4*a - 2*c_2*c_4^2*a + 4*a*c_2*c_0^2 - a*c_1^2*c_4 - a*c_1^2*c_0)

# Genericity assumptions: a+1, b+1, a+b+1 and x+y-1 are non-zero
gc = [(a+1)*w_0-1, (b+1)*w_1-1, (a+b+1)*w_2-1, (x+y-1)*w_3-1]

In [30]:
%%time

# Ideal of all relations:
I = ideal(Rel + Eq + Form + gc)

# Eliminate variables w_i, x, y
J = I.elimination_ideal([w_0,w_1,w_2,w_3,x,y])

# Extract variables of interest
g = J.gens()
dim = J.dimension()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.22 ms



In [37]:
def print_result():
    global dim
    global g
    if dim == -1:
        passed = False
    else:
        passed = True
    print 'Test passed:', passed, '\n'
    if passed:
        print 'Generators of J:\n', g

In [38]:
print_result()

Test passed: True 

Generators of J:
[b - 2, a - 4, c_5 - 3, c_4 + 1, c_3, c_2, c_1 - 2, c_0 - 1]
